In [3]:
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate


MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2         
MAX_LENGTH = 256
SEED = 12

# Load paper dataset
dataset = load_dataset("imdb")


# Rename split for consistency
dataset = {
    "train": dataset["train"],
    "validation": dataset["test"],  # paper reports test accuracy
}

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH,
    )

tokenized = {
    "train": dataset["train"].map(tokenize_fn, batched=True),
    "validation": dataset["validation"].map(tokenize_fn, batched=True),
}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Remove unused columns
keep_cols = ["input_ids", "attention_mask", "label"]
tokenized["train"] = tokenized["train"].remove_columns(
    [c for c in tokenized["train"].column_names if c not in keep_cols]
)
tokenized["validation"] = tokenized["validation"].remove_columns(
    [c for c in tokenized["validation"].column_names if c not in keep_cols]
)

# Rename labels
tokenized["train"] = tokenized["train"].rename_column("label", "labels")
tokenized["validation"] = tokenized["validation"].rename_column("label", "labels")


# Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
)


# Metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(
            predictions=preds,
            references=labels
        )["accuracy"]
    }


# Training Arguments
args = TrainingArguments( 
    output_dir="distilbert-imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=200,
    seed=SEED,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train & Evaluate
trainer.train()
print(trainer.evaluate())


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\cengi\AppData\Local\Temp\ipykernel_31128\2805329008.py:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
200,0.410400
400,0.293500
600,0.288400
800,0.278700
1000,0.270800
1200,0.268200
1400,0.280700
1600,0.235500
1800,0.176200
2000,0.163300


c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.3485184907913208, 'eval_accuracy': 0.914, 'eval_runtime': 843.8547, 'eval_samples_per_second': 29.626, 'eval_steps_per_second': 0.927, 'epoch': 3.0}


In [4]:
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate


MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 2         
MAX_LENGTH = 256
SEED = 12

# Load paper dataset
dataset = load_dataset("imdb")


# Rename split for consistency
dataset = {
    "train": dataset["train"],
    "validation": dataset["test"],  # paper reports test accuracy
}

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH,
    )

tokenized = {
    "train": dataset["train"].map(tokenize_fn, batched=True),
    "validation": dataset["validation"].map(tokenize_fn, batched=True),
}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Remove unused columns
keep_cols = ["input_ids", "attention_mask", "label"]
tokenized["train"] = tokenized["train"].remove_columns(
    [c for c in tokenized["train"].column_names if c not in keep_cols]
)
tokenized["validation"] = tokenized["validation"].remove_columns(
    [c for c in tokenized["validation"].column_names if c not in keep_cols]
)

# Rename labels
tokenized["train"] = tokenized["train"].rename_column("label", "labels")
tokenized["validation"] = tokenized["validation"].rename_column("label", "labels")


# Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
)


# Metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(
            predictions=preds,
            references=labels
        )["accuracy"]
    }


# TrainingArguments
args = TrainingArguments(
    output_dir="distilbert-imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=200,
    seed=SEED,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train & Evaluate
trainer.train()
print(trainer.evaluate())

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cengi\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\cengi\AppData\Local\Temp\ipykernel_31128\57357204.py:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
200,0.382100
400,0.284900
600,0.262300
800,0.262500
1000,0.251200
1200,0.245500
1400,0.254700
1600,0.218900
1800,0.156900
2000,0.136600


c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.34386318922042847, 'eval_accuracy': 0.92264, 'eval_runtime': 1740.2365, 'eval_samples_per_second': 14.366, 'eval_steps_per_second': 0.449, 'epoch': 3.0}
